In [1]:
import sys
sys.path.append('../../../../laddie/src/')
sys.path.append('../../../src2/')

import numpy as np
import xarray as xr
import pandas as pd
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo

from laddie import Laddie
from geometry3 import Geometry
from forcing import Forcing
from tools import im,jm
from tools import im,jm


%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [2]:
exp_class = os.getcwd()[64:-5]
run = os.getcwd()[-4:]
savename = f'{exp_class}_{run}'
print(f'Documents are saved to folder \'{os.getcwd()}\' with save name', savename)

Documents are saved to folder '/Users/5941962/Documents/GitHub/laddie-run-analysis/experiments/CD-Paolo/r001' with save name CD-Paolo_r001


In [ ]:
# Select input file geometry and initial geometry at t = 0
ds = xr.open_dataset('../../../data/Paolo-variablegeom/CD_paolo_30yrs.nc')
ds_t0 = ds.isel(t=0)

# Generate geometry
N = 2                          # standard resolution is at 500 m, N = 2: 1 km, N = 4: 2 km !!! DOES NOT WORK PROPERLY
geom = Geometry(ds_t0, savename)
geom.coarsen(N=N)
geom = geom.create()

# Select forcing
forc = Forcing(geom).isomip('WARM')

In [ ]:
# Set up LADDIE
laddie = Laddie(forc)

laddie.Ah = 6               #Horizontal viscosity
laddie.Kh = 1               #Horizontal diffusivity
laddie.ci = 0               #Assume insulation ice
laddie.slip = .5            #Semi-slip
laddie.convop = 1

laddie.correctisf = True    #Correct ice front
laddie.nnUV = 1             #Weight of nearest neighbours for new cells
laddie.vcut = 1             #maximum velocity

laddie.Cdtop = 2.5e-3       #Top drag coefficient
laddie.minD = 4             #Minimum layer thickness
laddie.dt = 120             #Time step
laddie.nu = .5
laddie.gamTfix = 0.00018

In [ ]:
laddiedays = 1 # number of days to integrate laddie for each ice time step
laddiespinup = 20 # number of days to spin up laddie

laddie.saveday = laddiedays
laddie.restday = 100

ds = laddie.compute(days=laddiespinup, savename=f'{os.getcwd()}/{savename}')